<a href="https://colab.research.google.com/github/tanmayispiddy07/gemma_model_phrase_recognization/blob/main/key_phrase_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y pandas
!pip install pandas==1.3.5
!pip uninstall -y transformers
!pip install transformers
!pip install huggingface_hub

Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 57.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.18.0 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
bigframes 1.15.0 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.3.5 which is incompatible.
geopandas 0.14.4 requires pandas>=1.4.0, but you have pandas 1.3.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 1.3.5 which is incompatible.
plotnine 0.12.4 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
statsmodels 0.14.2 requires pandas!=2.1.0,>=1.4, but you have pandas 1.3.5 which is incompa

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from huggingface_hub import login
import random
import pandas as pd

In [ ]:
from google.colab import userdata
api=userdata.get('secretName')

In [ ]:
login(token=api)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving case1_train.csv to case1_train (3).csv
User uploaded file "case1_train (3).csv" with length 9345445 bytes


In [ ]:
dataset = load_dataset('csv', data_files={'train': 'case1_train.csv', 'test': 'case1_train.csv'})

In [ ]:
random.seed(42)
subset_size = int(0.1 * len(dataset))
subset_indices = random.sample(range(len(dataset['train'])), subset_size)
small_train_dataset = dataset['train'].select(subset_indices)

In [ ]:
random.seed(42)
subset_size_test = int(0.05 * len(dataset))
subset_indices_test = random.sample(range(len(dataset['test'])), subset_size)
small_test_dataset = dataset['test'].select(subset_indices)

In [ ]:
base_model = "google/gemma-2-2b-it"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model).to(device)
tokenizer = AutoTokenizer.from_pretrained(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def format_prompt(example):
    text = example['Tweet']
    key_phrase = example['Target']
    prompt = f"Question: Extract the implicit key phrase from the given text.\nKey: {text}\nValue: {key_phrase}\n"
    return {'input_text': prompt}

In [ ]:
formatted_train_dataset = small_train_dataset.map(format_prompt)
formatted_test_dataset = small_test_dataset.map(format_prompt)

In [ ]:
def tokenize_function(example):
    return tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=128)

In [ ]:
tokenized_train_dataset = formatted_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = formatted_test_dataset.map(tokenize_function, batched=True)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=torch.cuda.is_available(),
    eval_strategy="epoch",
    remove_unused_columns=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained('./fine-tuned-model')

In [ ]:
tokenizer.save_pretrained('./fine-tuned-model')

In [ ]:
print("Model fine-tuned and saved successfully!")

In [ ]:
def zero_shot_inference(text):
    input_prompt = f"Question: Extract the implicit key phrase from the given text.\nKey: {text}\nValue:"
    input_ids = tokenizer.encode(input_prompt, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=50)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
unseen_data = [
    "Our souls continue after death, giving us an eternal existence.",
    "The rapid advancement of technology is shaping our future in unpredictable ways.",
    "Meditation brings peace to the mind and body."
]

In [ ]:
print("\nEvaluating model on unseen data:")
for text in unseen_data:
    key_phrase = zero_shot_inference(text)
    print(f"Text: {text}\nExtracted Key Phrase: {key_phrase}\n")

In [ ]:
from sklearn.metrics import classification_report
import numpy as np



In [ ]:
def evaluate_on_test_dataset(tokenizer, model, test_dataset, device):
    true_key_phrases = []                                                                                                                                                                                             predicted_key_phrases = []
    for example in test_dataset:
        text = example['Tweet']
        true_key_phrase = example['Target']


        predicted_key_phrase = zero_shot_inference(text)

        true_key_phrases.append(true_key_phrase)
        predicted_key_phrases.append(predicted_key_phrase)

    return true_key_phrases, predicted_key_phrases

In [ ]:
true_labels, predicted_labels = evaluate_on_test_dataset(tokenizer, model, formatted_test_dataset, device)

In [ ]:
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, zero_division=0))

In [ ]:
def format_prompt_stance(example):
    text = example['Tweet']
    stance = example['Stance']
    prompt = f"Question: What is the stance of the given text?\nText: {text}\nStance:"
    return {'input_text': prompt, 'true_stance': stance}

In [ ]:
formatted_train_dataset_stance = small_train_dataset.map(format_prompt_stance)
formatted_test_dataset_stance = small_test_dataset.map(format_prompt_stance)

In [ ]:
def zero_shot_inference_stance(text):
    input_prompt = f"Question: What is the stance of the given text?\nText: {text}\nStance:"
    input_ids = tokenizer.encode(input_prompt, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=50)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.strip()

In [ ]:
def evaluate_stance_detection(tokenizer, model, test_dataset, device):
    true_stances = []
    predicted_stances = []
    for example in test_dataset:
        text = example['Tweet']
        true_stance = example['true_stance']
        predicted_stance = zero_shot_inference_stance(text)

        true_stances.append(true_stance)
        predicted_stances.append(predicted_stance)

    return true_stances, predicted_stances

In [ ]:
true_stances, predicted_stances = evaluate_stance_detection(tokenizer, model, formatted_test_dataset_stance, device)

In [ ]:
print("Classification Report for Stance Detection:")
print(classification_report(true_stances, predicted_stances, zero_division=0))